In [3]:
!pip install kaggle unzip
!kaggle datasets download gauravduttakiit/loan-defaulter/
!unzip loan-defaulter

  Using cached kaggle-1.6.3-py3-none-any.whl
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for python-slugify from https://files.pythonhosted.org/packages/09/49/e05adaaa2d8604b7cfbce81af14c7a48c67d70a6e06cb47473c9673267db/python_slugify-8.0.2-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/78.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/78.2 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/78.2 kB ? eta -:--:--
     ----------------------------- -------- 61.4/78.2 kB 825.8 kB/s eta 0:00:01
     -------------------------------------- 78.2/78.2 kB 870.7 kB/s eta 0:00:00
  Created wheel for unzip: file


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  0%|          | 0.00/112M [00:00<?, ?B/s]
  1%|          | 1.00M/112M [00:00<00:12, 9.45MB/s]
  3%|▎         | 3.00M/112M [00:00<00:08, 14.1MB/s]
  4%|▍         | 5.00M/112M [00:00<00:07, 14.2MB/s]
  6%|▌         | 7.00M/112M [00:00<00:09, 12.2MB/s]
  8%|▊         | 9.00M/112M [00:00<00:09, 11.9MB/s]
 10%|▉         | 11.0M/112M [00:00<00:09, 11.4MB/s]
 12%|█▏        | 13.0M/112M [00:01<00:09, 11.4MB/s]
 13%|█▎        | 15.0M/112M [00:01<00:09, 10.9MB/s]
 15%|█▌        | 17.0M/112M [00:01<00:08, 12.4MB/s]
 17%|█▋        | 19.0M/112M [00:01<00:07, 13.5MB/s]
 19%|█▊        | 21.0M/112M [00:01<00:07, 13.0MB/s]
 20%|██        | 23.0M/112M [00:01<00:06, 14.2MB/s]
 22%|██▏       | 25.0M/112M [00:02<00:05, 15.6MB/s]
 24%|██▍       | 27.0M/112M [00:02<00:05, 16.1MB/s]
 26%|██▌       | 29.0M/112M [00:02<00:05, 15.7MB/s]
 28%|██▊       | 31.0M/112M [00:02<00:05, 16.2MB/s]
 29%|██▉       | 33.0M/112M [00:02<00:05, 15.9MB/s]
 31%|███       | 35.0M/112M [00:02<00:05, 15.3MB/s]
 33%|███▎      | 37.

In [6]:
import pandas as pd
import numpy as np

application_data = pd.read_csv('application_data.csv')
previous_application = pd.read_csv('previous_application.csv')
columns_description = pd.read_csv(r'columns_description.csv',skiprows=1)

In [7]:
mergeddf =  pd.merge(application_data,previous_application,on='SK_ID_CURR')

In [57]:
mergeddf_sample = mergeddf.sample(10000)

In [58]:
# count rows
mergeddf_sample.shape[0]

10000

In [59]:
# drop columns with more than 50% missing values
mergeddf_sample = mergeddf_sample.dropna(thresh=0.5*len(mergeddf_sample), axis=1)

# convert categorical columns to numerical
mergeddf_sample = pd.get_dummies(mergeddf_sample)

# convert all columns to float
mergeddf_sample = mergeddf_sample.astype(float)


In [61]:
# pipeline to drop na, impute missing values, filter by VIF, and normalize
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])



# get features and labels. drop target column
X = mergeddf_sample.drop(['TARGET'],axis=1)
X = num_pipeline.fit_transform(X)
mergeddf_sample_num = num_pipeline.fit_transform(mergeddf_sample)

y = mergeddf_sample['TARGET']

# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=42)

# train linear svc
from sklearn.svm import LinearSVC

lin_svc = LinearSVC()
lin_svc.fit(X_train,y_train)
y_pred = lin_svc.predict(X_test)

c:\Users\uzair\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\uzair\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [62]:
# accuracy score
from sklearn.metrics import accuracy_score
print(f'Accuracy score: {accuracy_score(y_test,y_pred)}')

# confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)



Accuracy score: 0.9125


array([[1817,   10],
       [ 165,    8]], dtype=int64)

In [ ]:
# store model weights with joblib